In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
from numpy.fft import fft

from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings('ignore')

plt.rcParams["figure.figsize"] = (30, 20)

## process_file() takes a string with a path to the .csv


def process_file(pathname):
    res = pd.read_csv(pathname, sep=',', header=0, index_col=None, names=[
                      'time', 'seconds', 'z', 'y', 'x'])
    return res


def butterworth(df):
    nyquist_frequency = 0.5 * 50

    low = 0.5 / nyquist_frequency
    high = 2.2 / nyquist_frequency

    b, a = signal.butter(3, Wn=[low, high], btype='bandpass')

    df['x'] = signal.filtfilt(b, a, df['x'])
    df['y'] = signal.filtfilt(b, a, df['y'])
    df['z'] = signal.filtfilt(b, a, df['z'])

    return df


In [8]:
slp_5mins_bw = process_file('processed_data\srp_5min_bw.csv')

dlp_upstairs_1_bw = process_file('processed_data/dlp_upstairs_1_bw.csv')
dlp_upstairs_2_bw = process_file('processed_data/dlp_upstairs_2_bw.csv')
dlp_downstairs_1_bw = process_file('processed_data/dlp_downstairs_1_bw.csv')
dlp_downstairs_2_bw = process_file('processed_data/dlp_downstairs_2_bw.csv')

mrp_upstairs_1_bw = butterworth(process_file(
    'processed_data/mrp_upstairs_16steps.csv'))
m_up24_1_bw = process_file('processed_data\m_up24_1_bw.csv')
m_up24_2_bw = process_file('processed_data\m_up24_2_bw.csv')
m_up24_3_bw = process_file('processed_data\m_up24_3_bw.csv')
m_up24_4_bw = process_file('processed_data\m_up24_4_bw.csv')
m_up24_5_bw = process_file('processed_data\m_up24_5_bw.csv')

mrp_downstairs_1 = butterworth(process_file(
    'processed_data/mrp_downstairs_16steps.csv'))
m_down24_1_bw = process_file('processed_data\m_down24_1_bw.csv')
m_down24_2_bw = process_file('processed_data\m_down24_2_bw.csv')
m_down24_3_bw = process_file('processed_data\m_down24_3_bw.csv')
m_down24_4_bw = process_file('processed_data\m_down24_4_bw.csv')
m_down24_5_bw = process_file('processed_data\m_down24_5_bw.csv')

dlp_upstairs_1_bw['movement'] = 'upstairs'
dlp_upstairs_2_bw['movement'] = 'upstairs'
dlp_downstairs_1_bw['movement'] = 'downstairs'
dlp_downstairs_2_bw['movement'] = 'downstairs'
mrp_upstairs_1_bw['movement'] = 'upstairs'
m_up24_1_bw['movement'] = 'upstairs'
m_up24_2_bw['movement'] = 'upstairs'
m_up24_3_bw['movement'] = 'upstairs'
m_up24_4_bw['movement'] = 'upstairs'
m_up24_5_bw['movement'] = 'upstairs'
mrp_downstairs_1['movement'] = 'downstairs'
m_down24_1_bw['movement'] = 'downstairs'
m_down24_2_bw['movement'] = 'downstairs'
m_down24_3_bw['movement'] = 'downstairs'
m_down24_4_bw['movement'] = 'downstairs'
m_down24_5_bw['movement'] = 'downstairs'

dlp_upstairs_1_split = np.array_split(dlp_upstairs_1_bw, 4)
dlp_upstairs_2_split = np.array_split(dlp_upstairs_2_bw, 4)
dlp_downstairs_1_split = np.array_split(dlp_downstairs_1_bw, 4)
dlp_downstairs_2_split = np.array_split(dlp_downstairs_2_bw, 4)
mrp_upstairs_1_split = np.array_split(mrp_upstairs_1_bw, 4)
m_up24_1_bw_split = np.array_split(m_up24_1_bw, 4)
m_up24_2_bw_split = np.array_split(m_up24_2_bw, 4)
m_up24_3_bw_split = np.array_split(m_up24_3_bw, 4)
m_up24_4_bw_split = np.array_split(m_up24_4_bw, 4)
m_up24_5_bw_split = np.array_split(m_up24_5_bw, 4)
mrp_downstairs_1_split = np.array_split(mrp_downstairs_1, 4)
m_down24_1_bw_split = np.array_split(m_down24_1_bw, 4)
m_down24_2_bw_split = np.array_split(m_down24_2_bw, 4)
m_down24_3_bw_split = np.array_split(m_down24_3_bw, 4)
m_down24_4_bw_split = np.array_split(m_down24_4_bw, 4)
m_down24_5_bw_split = np.array_split(m_down24_5_bw, 4)

tmove = []
tdata = []

for i in [dlp_upstairs_1_split, dlp_upstairs_2_split, dlp_downstairs_1_split, dlp_downstairs_2_split, mrp_upstairs_1_split, m_up24_1_bw_split, m_up24_2_bw_split, m_up24_3_bw_split, m_up24_4_bw_split, m_up24_5_bw_split, mrp_downstairs_1_split, m_down24_1_bw_split, m_down24_2_bw_split, m_down24_3_bw_split, m_down24_4_bw_split, m_down24_5_bw_split]:
    for j in range(len(i) - 1):
        temp = i[j]

        # if ((temp['movement'] == 'walking').all()):
            # tmove.append('walking')
        if ((temp['movement'] == 'upstairs').all()):
            tmove.append('upstairs')
        if ((temp['movement'] == 'downstairs').all()):
            tmove.append('downstairs')

        temp['combined'] = temp['y'] + temp['z']
        tdata.append(np.asarray(temp['combined']))

training_data = pd.DataFrame(tmove)
training_data['joined'] = pd.Series(tdata, index=training_data.index)
training_data

training_data.rename(columns={0: 'movement'}, inplace=True)
training_data = pd.concat([training_data['movement'], training_data.pop(
    'joined').apply(pd.Series)], axis=1).dropna(axis=1)
training_data


,movement,0,1,2,3,4,5,6,7,8,...,83,84,85,86,87,88,89,90,91,92
0,upstairs,0.939890,1.336056,1.687374,1.960024,2.125367,2.161996,2.057994,1.813111,1.440323,...,-1.528790,-1.983224,-2.388069,-2.717393,-2.941399,-3.030885,-2.964149,-2.733299,-2.347548,-1.833174
1,upstairs,-2.918936,-2.707874,-2.313125,-1.782125,-1.175569,-0.555401,0.024901,0.526104,0.925730,...,0.609138,0.027509,-0.531105,-1.027495,-1.426952,-1.705235,-1.852168,-1.872014,-1.780947,-1.602765
2,upstairs,-0.389383,-0.614847,-0.869202,-1.126962,-1.356867,-1.528410,-1.618357,-1.614542,-1.516473,...,3.161443,2.990278,2.693271,2.264739,1.714643,1.068741,0.364760,-0.353732,-1.043302,-1.664654
3,upstairs,0.939890,1.336056,1.687374,1.960024,2.125367,2.161996,2.057994,1.813111,1.440323,...,-1.528790,-1.983224,-2.388069,-2.717393,-2.941399,-3.030885,-2.964149,-2.733299,-2.347548,-1.833174
4,upstairs,-2.918936,-2.707874,-2.313125,-1.782125,-1.175569,-0.555401,0.024901,0.526104,0.925730,...,0.609138,0.027509,-0.531105,-1.027495,-1.426952,-1.705235,-1.852168,-1.872014,-1.780947,-1.602765
5,upstairs,-0.389383,-0.614847,-0.869202,-1.126962,-1.356867,-1.528410,-1.618357,-1.614542,-1.516473,...,3.161443,2.990278,2.693271,2.264739,1.714643,1.068741,0.364760,-0.353732,-1.043302,-1.664654
6,downstairs,0.080866,0.352508,0.617629,0.867531,1.095384,1.296104,1.465948,1.602341,1.703949,...,1.816939,2.555747,3.198375,3.724863,4.108928,4.317218,4.311985,4.057262,3.527397,2.715994
7,downstairs,5.564241,5.680499,5.548802,5.207350,4.695917,4.049836,3.296113,2.452716,1.531608,...,4.106167,4.411691,4.465755,4.256439,3.782669,3.056238,2.103895,0.968590,-0.291257,-1.604722
8,downstairs,6.419121,5.619104,4.574549,3.354716,2.029495,0.665643,-0.674315,-1.931404,-3.049568,...,1.028760,-0.285185,-1.667172,-3.020960,-4.237730,-5.210101,-5.848207,-6.092861,-5.922182,-5.351339
9,downstairs,-1.260977,-0.749933,-0.224540,0.316197,0.860835,1.383520,1.844646,2.194058,2.376370,...,2.391580,3.634947,4.589428,5.217872,5.504445,5.451302,5.074712,4.401395,3.466142,2.311548


In [7]:
## ML portion starts here

X = training_data.drop(columns=['movement'])
y = training_data['movement']

X_train, X_valid, y_train, y_valid = train_test_split(X, y)

bayes_model = make_pipeline(
    GaussianNB()
)

knn_model = make_pipeline(
    StandardScaler(),
    KNeighborsClassifier(n_neighbors=8)
)

nn_model = make_pipeline(
    StandardScaler(),
    MLPClassifier(solver='lbfgs', hidden_layer_sizes=(
        16, 8, 4), activation='logistic', max_iter=100000)
)

dt_model = make_pipeline(
    StandardScaler(),
    DecisionTreeClassifier(max_depth=125)
)

rf_model = make_pipeline(
    RandomForestClassifier(n_estimators=1500, max_depth=3, min_samples_leaf=5)
)

en_model = make_pipeline(
    VotingClassifier([
        ('nb', GaussianNB()),
        ('knn', KNeighborsClassifier(8)),
        ('tree1', DecisionTreeClassifier(max_depth=20)),
        ('tree2', DecisionTreeClassifier(min_samples_leaf=4)),
        ('rf', RandomForestClassifier(
            n_estimators=250, max_depth=15, min_samples_leaf=2)),
        ('mlp', MLPClassifier(solver='lbfgs', hidden_layer_sizes=(16, 8, 6), activation='logistic', max_iter=100000))])

)

print('bayes')
bayes_model.fit(X_train, y_train)
print(bayes_model.score(X_train, y_train))
print(bayes_model.score(X_valid, y_valid))
y_predicted = bayes_model.predict(X_valid)
print(classification_report(y_valid, y_predicted))

print('')
print('knn')
knn_model.fit(X_train, y_train)
print(knn_model.score(X_train, y_train))
print(knn_model.score(X_valid, y_valid))
y_predicted = knn_model.predict(X_valid)
print(classification_report(y_valid, y_predicted))

print('')
print('neural net')
nn_model.fit(X_train, y_train)
print(nn_model.score(X_train, y_train))
print(nn_model.score(X_valid, y_valid))
y_predicted = nn_model.predict(X_valid)
print(classification_report(y_valid, y_predicted))

print('')
print('decision tree')
dt_model.fit(X_train, y_train)
print(dt_model.score(X_train, y_train))
print(dt_model.score(X_valid, y_valid))
y_predicted = dt_model.predict(X_valid)
print(classification_report(y_valid, y_predicted))

print('')
print('random forest')
rf_model.fit(X_train, y_train)
print(rf_model.score(X_train, y_train))
print(rf_model.score(X_valid, y_valid))
y_predicted = rf_model.predict(X_valid)
print(classification_report(y_valid, y_predicted))

print('')
print('ensemble')
en_model.fit(X_train, y_train)
print(en_model.score(X_train, y_train))
print(en_model.score(X_valid, y_valid))
y_predicted = en_model.predict(X_valid)
print(classification_report(y_valid, y_predicted))


bayes
0.7777777777777778
0.9166666666666666
              precision    recall  f1-score   support

  downstairs       1.00      0.83      0.91         6
    upstairs       0.86      1.00      0.92         6

    accuracy                           0.92        12
   macro avg       0.93      0.92      0.92        12
weighted avg       0.93      0.92      0.92        12


knn
0.75
0.9166666666666666
              precision    recall  f1-score   support

  downstairs       1.00      0.83      0.91         6
    upstairs       0.86      1.00      0.92         6

    accuracy                           0.92        12
   macro avg       0.93      0.92      0.92        12
weighted avg       0.93      0.92      0.92        12


neural net
1.0
0.5833333333333334
              precision    recall  f1-score   support

  downstairs       0.57      0.67      0.62         6
    upstairs       0.60      0.50      0.55         6

    accuracy                           0.58        12
   macro avg       0